In [1]:
import requests
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QPushButton, QVBoxLayout, QTextEdit
from PyQt5.QtCore import Qt

In [4]:
api_key = 'your_api_key'

# Set the endpoint
endpoint = "http://ws.audioscrobbler.com/2.0/"

# Set parameters for the top tracks API call
top_tracks_params = {
    "method": "chart.getTopTracks",
    "api_key": api_key,
    "format": "json",
    "limit": 500  # Set the limit to the number of tracks you want
}

# Make the top tracks API call
top_tracks_response = requests.get(endpoint, params=top_tracks_params)
top_tracks_data = top_tracks_response.json()

# Create a list to store track information
tracks_list = []

# Iterate through the top tracks and get detailed information
for track_info in top_tracks_data['tracks']['track']:
    track_name = track_info['name']
    artist_name = track_info['artist']['name']

    # Set parameters for the track.getTopTags API call
    track_tags_params = {
        "method": "track.getTopTags",
        "api_key": api_key,
        "format": "json",
        "track": track_name,
        "artist": artist_name
    }

    # Make the track.getTopTags API call
    track_tags_response = requests.get(endpoint, params=track_tags_params)
    track_tags_data = track_tags_response.json()

    # Check if track tags information is available
    if 'toptags' in track_tags_data and 'tag' in track_tags_data['toptags']:
        # Extracting tag names from the response
        tags = [tag['name'] for tag in track_tags_data['toptags']['tag']]
        
        # Append track information to the list, including tags
        tracks_list.append({
            'Track': track_name,
            'Artist': artist_name,
            'Tags': tags,
            'Duration': track_info['duration'],
            'Playcount': track_info['playcount'],
            'Listeners': track_info['listeners'],
            'URL': track_info['url']
        })
    else:
        # If tags are not available, append with None
        tracks_list.append({
            'Track': track_name,
            'Artist': artist_name,
            'Tags': None,
            'Duration': track_info['duration'],
            'Playcount': track_info['playcount'],
            'Listeners': track_info['listeners'],
            'URL': track_info['url']
        })

# Create a DataFrame from the list
tracks_df = pd.DataFrame(tracks_list)

# Save the DataFrame to a CSV file
tracks_df.to_csv('top_tracks_tags_info.csv', index=False)


KeyboardInterrupt: 

In [2]:
df = pd.read_csv('top_tracks_tags_info.csv')
df = df.reset_index()
df = df.rename(columns={'index': 'ID'})
df['Track'] = df['Track'].apply(lambda x: x.lower())
df['Duration'] = df['Duration'].astype(int)
df['Playcount'] = df['Playcount'].astype(int)
df['Listeners'] = df['Listeners'].astype(int)
df.head()

,ID,Track,Artist,Tags,Duration,Playcount,Listeners,URL
0,0,my love mine all mine,Mitski,"['singer-songwriter', 'soft rock', 'Traditiona...",0,7291767,489444,https://www.last.fm/music/Mitski/_/My+Love+Min...
1,1,paint the town red,Doja Cat,"['pop', 'rap', 'american', 'rnb', 'trap', 'FM4...",0,6068612,585715,https://www.last.fm/music/Doja+Cat/_/Paint+the...
2,2,cruel summer,Taylor Swift,"['electropop', 'synthpop', 'pop', 'industrial ...",0,27142593,947082,https://www.last.fm/music/Taylor+Swift/_/Cruel...
3,3,vampire,Olivia Rodrigo,"['pop rock', 'pop', 'piano rock', 'guts', 'sin...",0,12763501,711035,https://www.last.fm/music/Olivia+Rodrigo/_/vam...
4,4,pink + white,Frank Ocean,"['rnb', 'MySpotigramBot', 'soul', 'Neo-Soul', ...",0,26773230,1297121,https://www.last.fm/music/Frank+Ocean/_/Pink+%...


In [3]:
df['tags_str'] = df['Tags'].apply(lambda x: ' '.join(x) if x else '')

# Combine relevant features into a text field for TF-IDF
df['features'] = df['Track'] + ' ' + df['Artist'] + ' ' + df['tags_str']

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['features'])

# Calculate the cosine similarity between tracks
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert the cosine similarities to a DataFrame
cosine_similarities_df = pd.DataFrame(cosine_similarities, index=df.index, columns=df.index)

# Function to get top N similar tracks for a given track
def get_top_similar_tracks(track_index, top_n=5):
    similar_tracks_indices = cosine_similarities_df[track_index].sort_values(ascending=False)[1:top_n+1].index
    similar_tracks_info = df.loc[similar_tracks_indices, ['Track', 'Artist', 'URL']]
    
    return similar_tracks_info

# Function to recommend tracks based on similar tracks
def recommend_tracks(track_name, top_n=5):
    track_indices = df.index[df['Track'] == track_name].tolist()
    if not track_indices:
        print(f"Track '{track_name}' not found in the DataFrame.")
        return pd.DataFrame(columns=['Track', 'Artist', 'URL'])  # Return an empty DataFrame
    track_index = track_indices[0]
    similar_tracks_info = get_top_similar_tracks(track_index, top_n=top_n)

    return similar_tracks_info

# Example: Recommend top 3 tracks similar to a given track
track_to_recommend = 'Cruel Summer'
recommendations = recommend_tracks(track_to_recommend, top_n=3)

print(f"Top Recommendations for Track '{track_to_recommend}':")
print(recommendations)



Track 'Cruel Summer' not found in the DataFrame.
Top Recommendations for Track 'Cruel Summer':
Empty DataFrame
Columns: [Track, Artist, URL]
Index: []


In [4]:
class MusicRecommendationGUI(QWidget):
    def __init__(self):
        super().__init__()

        # Initialize GUI components
        self.init_ui()

    def init_ui(self):
        # Create QLabel, QLineEdit, QPushButton, and QTextEdit
        self.track_label = QLabel('Enter Track Name:')
        self.track_input = QLineEdit()
        self.recommend_button = QPushButton('Get Recommendations')
        self.recommend_output = QTextEdit()

        # Connect the button click event to the recommendation function
        self.recommend_button.clicked.connect(self.show_recommendations)

        # Set up the layout
        layout = QVBoxLayout()
        layout.addWidget(self.track_label)
        layout.addWidget(self.track_input)
        layout.addWidget(self.recommend_button)
        layout.addWidget(self.recommend_output)

        # Set the main layout for the window
        self.setLayout(layout)

        # Set window properties
        self.setWindowTitle('Music Recommendation System')
        self.setGeometry(100, 100, 400, 300)

        # Show the window
        self.show()

    def show_recommendations(self):
        # Get the track name from the input
        track_to_recommend = self.track_input.text()

        # Call your recommendation function (replace with your actual function)
        recommendations = recommend_tracks(track_to_recommend, top_n=3)

        # Display recommendations in the QTextEdit
        self.recommend_output.clear()
        self.recommend_output.setPlainText(f"Top Recommendations for Track '{track_to_recommend}':\n{recommendations.to_string(index=False)}")


if __name__ == '__main__':
    app = QApplication(sys.argv)
    gui = MusicRecommendationGUI()
    sys.exit(app.exec_())


Track 'calm down' not found in the DataFrame.


SystemExit: 0

c:\Users\Bhavani Patel\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
